# Llama using llama-cpp library

In [3]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -I llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install -I llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install accelerate

Using pip 23.3.1 from /home/dephinate/miniconda3/envs/medicalChatBot/lib/python3.9/site-packages/pip (python 3.9)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.2 MB/s eta 0:00:002.1 MB/s eta 0:00:01
  Installing build dependencies ...   Running command pip subprocess to install build dependencies
    Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl.metadata (14 kB)
    Using cached cmake-3.29.0.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.1 kB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
    Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
    Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
    Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
    Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
  Using cached s

In [5]:
from tqdm.autonotebook import tqdm as notebook_tqdm

In [6]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [6]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [10]:
#  To Download the model
# model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [11]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

llama.cpp: loading model from /home/dephinate/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_i

In [12]:
prompt = "Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''


In [13]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

KeyboardInterrupt: 

# Llama using Langchain
* Use 7B parameter model

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [1]:
# Login to huggingface account
# !huggingface-cli login

In [10]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
from langchain.llms import huggingface_pipeline

In [7]:
from transformers import AutoTokenizer
import transformers
import torch
from tqdm.auto import tqdm

In [13]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
tokenizer.eos_token

'</s>'

In [ ]:
pipeline  = transformers.pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
    device_map = "auto",
    max_length = 1000,
    do_sample = True,
    top_k = 10,
    num_return_sequences = 1, # For chatbots, determines how many alternative sequences you want the model to generate. 
    eos_token_id = tokenizer.eos_token_id

)

In [ ]:
llm = huggingface_pipeline(pipeline= pipeline, model_kwargs={'temperature':0})
prompt = "Is God real, don not think about hurting sentiments, give honest answer"
print(llm(prompt))

Prompt Templates    
When we are making a user application, it is not ideal to send the user input directly to the llm. Instead we use prompt templates.
LangChain facilitates prompt management and optimization

In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [24]:
prompt_template = PromptTemplate(
    input_variables = ["book"],
    template = "Give summary of the book {book}"
)

prompt = prompt_template.format(book="Stillness is the key")
prompt

'Give summary of the book Stillness is the key'

In [ ]:
chain = LLMChain(llm=llm, prompt= prompt)
response = chain.run("Wings of Fire")